In [1]:
# bob.refresh {"cron": "5,30 2-12 * * *", "timeout": 40000}

In [2]:
%run __init__.ipynb

     |████████████████████████████████| 141 kB 24.8 MB/s eta 0:00:01
Successfully connected to MongoDB


In [3]:
hc = bob.HealthCheck()
hc_key = "51c8f3e8-3d84-4888-878e-a53f0a152e1a"
if USE_HEALTHCHECKS:
    step_1 = hc.start(hc_key)
    print(step_1)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/51c8f3e8-3d84-4888-878e-a53f0a152e1a, 2020-06-12


In [4]:
%%time

#--- Check if the data are already updated
def init_script():
    stop = False
    file_path = os.path.join(OUTPUT_FOLDER,'WORLD_DB_TREND.csv')
    if path.exists(file_path):
        df_check = pd.read_csv(file_path, sep=';',low_memory=False)
        date_check = df_check[df_check['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y')]
        if len(date_check) == 0:
            %run "covid-19_world.ipynb"
        else:
            if USE_HEALTHCHECKS:
                step_2 = hc.done(hc_key)
                print(step_2)
                print(f'Data already updated !')
                stop = True
    else:
        %run "covid-19_world.ipynb"
    return stop
    
stop = init_script()
if stop:
    %stop

WORLD_CONFIRMED successfully saved in WORLD_CONFIRMED
Dataframe WORLD_CONFIRMED successfully save in database covid-19 in MongoDB. Time: --- 0.0960395336151123 secnds ---
WORLD_DEATHS successfully saved in WORLD_DEATHS
Dataframe WORLD_DEATHS successfully save in database covid-19 in MongoDB. Time: --- 0.09109067916870117 secnds ---
WORLD_RECOVERED successfully saved in WORLD_RECOVERED
Dataframe WORLD_RECOVERED successfully save in database covid-19 in MongoDB. Time: --- 0.08646345138549805 secnds ---
US_CONFIRMED successfully saved in US_CONFIRMED
Dataframe US_CONFIRMED successfully save in database covid-19 in MongoDB. Time: --- 0.8723409175872803 secnds ---
US_DEATHS successfully saved in US_DEATHS
Dataframe US_DEATHS successfully save in database covid-19 in MongoDB. Time: --- 0.8876199722290039 secnds ---
CPU times: user 2.19 s, sys: 47.5 ms, total: 2.24 s
Wall time: 2.75 s
CPU times: user 1.23 s, sys: 78.7 ms, total: 1.3 s
Wall time: 1.46 s
WORLD_DB_CONCAT successfully saved in WO

In [5]:
%%time

#--- Load data source
db_conso = get_datasource('WORLD_DB_CONSO')
db_all = get_datasource('WORLD_DB_ALL')
db_trend = get_datasource('WORLD_DB_TREND')

CPU times: user 3.6 s, sys: 6.45 s, total: 10.1 s
Wall time: 10.5 s


In [6]:
def df_to_csv_mongo(df,df_name):
    if not df.empty:
        # Save in CSV
        df_save(df, df_name,'csv')
        
        # Save in MongoDB
        if USE_MONGO:
            bob.mongo.save_df(df, df_name, DB_APP,True)
    else:
        print(f'DataFrame {df_name} is empty !')

In [7]:
%%time

#Dashboard value dynamic template
def dashboard_value(df,kpi,domain_num):
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain = df[(df['ENTITY_GROUPS'].isin(filters)) & (df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['KPI'],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
    })
    domain = domain.sort_values(by=['ENTITY','SCENARIO'],ascending=False)
    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),("SCENARIO", DESCENDING)])
    return domain

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [8]:
#Duplicate to rank countries WORLDWIDE
def get_ww(df): 
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df_ww = df.copy()[(df['ENTITY'].isin(filters))] 
    df_ww['ENTITY'] = "WORLDWIDE"           
    df = df.append(df_ww)
    return df

In [9]:
#Dashboard leaderboard template
def dashboard_leaderboard(df,kpi,domain_num):
    domain = df[(df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['ENTITY'],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
    })
    domain = domain.sort_values(by=['ENTITY','SCENARIO'],ascending=False)
    if domain_num == '010' or domain_num == '011' or domain_num == '012':
        domain = get_ww(domain)
    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),("SCENARIO", DESCENDING)])
    return domain

In [10]:
%%time

#-- Date selector : ENTITIES
def get_entities(df):
    cols_to_rename = {"ENTITY": "ENTITIES", "ENTITY_GROUPS": "GROUPS"}
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df = df[df['ENTITY_GROUPS'].isin(filters)].rename(index=str, columns=cols_to_rename)[['ENTITIES','GROUPS']].drop_duplicates().sort_values(by="ENTITIES", ascending=True).reset_index(drop=True)
    df['ORDER'] = df.index + 8
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER'] = 8
    
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER_GROUPS'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER_GROUPS'] = 8

    df.loc[df['GROUPS'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['GROUPS'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['GROUPS'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['GROUPS'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['GROUPS'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['GROUPS'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['GROUPS'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['GROUPS'] == 'To be affected', 'ORDER_GROUPS'] = 8
    
    df.loc[df['GROUPS'] == 'WORLDWIDE','ENTITY_DISPLAY'] = df['ENTITIES']
    df.loc[df['GROUPS'] != 'WORLDWIDE','ENTITY_DISPLAY'] = '--- ' + df['ENTITIES']
    return df.sort_values(by=['ORDER_GROUPS','ORDER'], ascending=True)

entities = get_entities(db_conso)
df_to_csv_mongo(entities,'ENTITIES')
# entities

ENTITIES successfully saved in ENTITIES
Dataframe ENTITIES successfully save in database app-wsr in MongoDB. Time: --- 0.03865671157836914 secnds ---
CPU times: user 91 ms, sys: 2.26 ms, total: 93.3 ms
Wall time: 128 ms


In [11]:
%%time

#-- Date selector : SCENARIOS
def get_scenarios(df):
    df = df[['SCENARIO','DATE_ORDER']].drop_duplicates()
    df['SCENARIO_DIPLAY'] = pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y').dt.strftime('%d %b %Y')
    df['SCENARIO_TODAY'] = (pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y') + timedelta(days = 1)).dt.strftime('%d/%m/%Y')
    df['SCENARIO_TODAY_DIPLAY'] = pd.to_datetime(df['SCENARIO_TODAY'], format='%d/%m/%Y').dt.strftime('%d %b %Y')
    return df.sort_values(by='DATE_ORDER', ascending=False)

scenarios = get_scenarios(db_all)
df_to_csv_mongo(scenarios,'SCENARIOS')
# scenarios

SCENARIOS successfully saved in SCENARIOS
Dataframe SCENARIOS successfully save in database app-wsr in MongoDB. Time: --- 0.028421878814697266 secnds ---
CPU times: user 72.9 ms, sys: 6.6 ms, total: 79.5 ms
Wall time: 105 ms


In [12]:
%%time

#--- DASHBOARD ---
#-- VALUE DYNAMIC KPIS
domain001 = dashboard_value(db_all,'Confirmed','001')
domain002 = dashboard_value(db_all,'Deaths','002')
domain003 = dashboard_value(db_all,'Recovered','003')
domain004 = dashboard_value(db_all,'Active cases','004')
domain005 = dashboard_value(db_all,"Fatality Rate",'005')
domain006 = dashboard_value(db_all,"Recovery Rate",'006')

001 successfully saved in 001
Dataframe 001 successfully save in database app-wsr in MongoDB. Time: --- 22.22281265258789 secnds ---
002 successfully saved in 002
Dataframe 002 successfully save in database app-wsr in MongoDB. Time: --- 20.957187175750732 secnds ---
003 successfully saved in 003
Dataframe 003 successfully save in database app-wsr in MongoDB. Time: --- 20.90709114074707 secnds ---
004 successfully saved in 004
Dataframe 004 successfully save in database app-wsr in MongoDB. Time: --- 21.797520875930786 secnds ---
005 successfully saved in 005
Dataframe 005 successfully save in database app-wsr in MongoDB. Time: --- 21.26444911956787 secnds ---
006 successfully saved in 006
Dataframe 006 successfully save in database app-wsr in MongoDB. Time: --- 22.330798387527466 secnds ---
CPU times: user 1min 6s, sys: 7.8 s, total: 1min 14s
Wall time: 2min 11s


In [13]:
%%time

#-- LEADERBOARD BY COUNTRIES
db_c = db_all[(db_all['ENTITY_GROUPS'] != 'WORLDWIDE')]
domain010 = dashboard_leaderboard(db_c,'Confirmed','010')
domain011 = dashboard_leaderboard(db_c,'Deaths','011')
domain012 = dashboard_leaderboard(db_c,'Recovered','012')

010 successfully saved in 010
Dataframe 010 successfully save in database app-wsr in MongoDB. Time: --- 21.120256900787354 secnds ---
011 successfully saved in 011
Dataframe 011 successfully save in database app-wsr in MongoDB. Time: --- 20.876609802246094 secnds ---
012 successfully saved in 012
Dataframe 012 successfully save in database app-wsr in MongoDB. Time: --- 21.220457553863525 secnds ---
CPU times: user 39.3 s, sys: 3.71 s, total: 43 s
Wall time: 1min 7s


In [14]:
%%time

#-- Covid-19 - TRENDS
def domain_101(df,domain_num):
#     filters = ['Confirmed','Deaths','Recovered','Active cases','Deaths ratio','Recovered ratio']
    filters = ['Confirmed','Deaths','Recovered','Active cases']
    domain = df[df['KPI'].isin(filters)]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'GROUPS': domain['KPI'],
        'BOTTOM_FILTER': domain['DATE_SCENARIO'],
        'UPPER_FILTER_M': domain['METRIC'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
        'LAST_UPDATE': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
    })

    #Deconfinement France
    groups_name = 'Deconfinement'
    df_france = domain.copy()[(domain['ENTITY'] == 'France') & (domain['GROUPS'] == 'Confirmed') & (domain['UPPER_FILTER_M'] == 'VARV')]
    df_france.loc[:,'GROUPS'] = groups_name
    df_france.loc[:,'VALUE'] = 3000
    domain = pd.concat([domain,df_france],axis=0)
    
    #Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases',groups_name])),'UPPER_FILTER_M'] = 'DAILY VARIATION (CASES)'
#     domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = 'DAILY VARIATION (PTS)'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases',groups_name])),'UPPER_FILTER_M'] = 'DAILY VARIATION (%)'
#     domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = ''
    
    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("UPPER_FILTER_M", DESCENDING), ("BOTTOM_FILTER", DESCENDING)])
    return domain

domain101 = domain_101(db_trend,'101')
# domain101

101 successfully saved in 101
Dataframe 101 successfully save in database app-wsr in MongoDB. Time: --- 85.58711695671082 secnds ---
CPU times: user 1min 10s, sys: 5.45 s, total: 1min 15s
Wall time: 1min 41s


In [15]:
%%time

#-- Covid-19 - TRENDS BY COUNTRIES
def domain_102(df,domain_num):
    domain = df.copy()
    max_date = domain['DATE_ORDER'].max()
    
    filters =  ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    countries_df = domain[(domain['ENTITY_GROUPS'].isin(filters)) & (domain['DATE_ORDER'] == max_date) & (domain['GROUPS'] == 'Confirmed') & (domain['BOTTOM_FILTER'] == 'Since beginning')].sort_values(by= 'VALUE', ascending=False).drop_duplicates().reset_index(drop=True)[:10]#['ENTITY'].drop_duplicates().values.tolist()[:10]
    countries_df['ENTITY_ORDER'] = countries_df.index
    countries_df = countries_df[['ENTITY','ENTITY_ORDER']]
    
    countries = countries_df['ENTITY'].values.tolist()
    
    domain = domain[(domain['ENTITY_GROUPS'] != "WORLDWIDE") & (domain['ENTITY'].isin(countries))]
    domain = pd.merge(domain,countries_df, on=['ENTITY'], how='left')

    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("GROUPS", ASCENDING), ("UPPER_FILTER_M", DESCENDING), ("BOTTOM_FILTER", DESCENDING)])
    return domain
    
domain102 = domain_102(domain101,'102')
# domain102

102 successfully saved in 102
Dataframe 102 successfully save in database app-wsr in MongoDB. Time: --- 17.918198823928833 secnds ---
CPU times: user 10.1 s, sys: 1.09 s, total: 11.2 s
Wall time: 19 s


In [16]:
%%time

#-- Covid-19 - RANKING BY COUNTRIES
#-> Referentials
ref_continent = pd.read_excel(REF_WSR_PATH, sheet_name='REF_CONTINENT')
def domain_103(df,ref,domain_num):
    domain = df.copy()
    cols_to_rename = {"LABEL": "KPI","ENTITY": "LABEL", "ENTITY_GROUPS":"ENTITY", "VARV":"VAR"}
    domain = get_ww(domain.drop(['DATE','UNIT_VAR','UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP'],axis=1).rename(index=str, columns=cols_to_rename).reset_index(drop=True))
    domain = pd.merge(domain, ref, left_on=['LABEL'], right_on=['COUNTRY_REGION'], how='left').drop(['CONTINENT','COUNTRY_REGION'],axis=1).reset_index(drop=True)

    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("SCENARIO", DESCENDING), ("LABEL", DESCENDING), ("KPI", ASCENDING), ("VALUE", DESCENDING)])
    return domain

domain103 = domain_103(db_all,ref_continent,'103')
# domain103

103 successfully saved in 103
Dataframe 103 successfully save in database app-wsr in MongoDB. Time: --- 61.581942081451416 secnds ---
CPU times: user 45.4 s, sys: 2.63 s, total: 48 s
Wall time: 1min 10s


In [17]:
%%time

#-- Covid-19 - DAILY TREND
def domain_106(df,domain_num):
    #-- Exclude Rates
    df = df[~df['KPI'].isin(['Fatality Rate',"Recovery Rate"])]
    
    #-- Calculate Value
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain_val = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VALUE','VALUE_D-1']][df['ENTITY_GROUPS'].isin(filters)]#[df['ENTITY_GROUPS'] == 'WORLDWIDE']
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI']
    domain_val = domain_val.melt(id_vars=cols_to_keep, value_name='VALUE',var_name='METRIC')
    domain_val['ENTITY_GROUPS'] = domain_val['ENTITY']
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'LABEL_ORDER'] = 99999999
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'LABEL_ORDER'] = 0
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'METRIC'] = domain_val['SCENARIO']
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'METRIC'] = (pd.to_datetime(domain_val['SCENARIO'], format='%d/%m/%Y') + timedelta(days=-1)).dt.strftime('%d/%m/%Y')
    
    cols_to_rename = {'METRIC':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_val = domain_val.drop('ENTITY',axis=1).rename(index=str, columns=cols_to_rename)
    domain_val['PRECISION'] = ',.0f'

    #--Variation
    #filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain_var = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VARV','VARP']][df['ENTITY_GROUPS']!='WORLDWIDE']#[df['ENTITY_GROUPS'].isin(filters)]
    cols_to_rename = {'VARV':'VALUE', 'VARP': 'VAR'}
    domain_var = domain_var.rename(index=str, columns=cols_to_rename).reset_index(drop=True)
    domain_var = domain_var[domain_var['VALUE'] !=0 ]
    
    #-- Get the first 10
    filters = domain_var[['ENTITY_GROUPS','SCENARIO','KPI']].drop_duplicates().values.tolist()
    domain_w = pd.DataFrame()
    for f in filters:
        tmp_df = domain_var[(domain_var['ENTITY_GROUPS'] == f[0]) & (domain_var['SCENARIO'] == f[1]) & (domain_var['KPI'] == f[2])].sort_values(by=["VALUE","ENTITY"],ascending=False).reset_index(drop=True)
        tmp_df['LABEL_ORDER'] = tmp_df.index.astype(int) + 1
        if len(tmp_df) > 9:
            tmp_df.loc[tmp_df.index > 9, 'ENTITY'] = 'Others'
            tmp_df.loc[tmp_df.index > 9, 'LABEL_ORDER'] = 11
            cols_to_group = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','LABEL_ORDER']
            tmp_df = tmp_df.groupby(cols_to_group, as_index=False).agg({'VALUE':'sum'}).reset_index(drop=True)
        tmp_df['PRECISION'] = '+,.0f'
        domain_w = domain_w.append(tmp_df)
        
    cols_to_rename = {'ENTITY':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_w = domain_w.rename(index=str, columns=cols_to_rename)
    domain_w['GROUPS'] = domain_w['LABEL']
    
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    cols_to_rename = {'LABEL':'GROUPS', 'ENTITY':'LABEL'}
    domain_w2 = domain_w.copy()[domain_w['ENTITY'].isin(filters)].drop('GROUPS',axis=1).rename(index=str, columns=cols_to_rename)
    domain_w2['ENTITY'] = 'WORLDWIDE'
    
    domain = pd.concat([domain_val,domain_w,domain_w2],axis=0)

    #-- Save df
    df_to_csv_mongo(domain,domain_num)
    df_collection = MONGO_DB[DB_APP][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("SCENARIO", DESCENDING), ("UPPER_FILTER_R", ASCENDING)])
    return domain

domain106 = domain_106(db_all,'106')
# domain106

106 successfully saved in 106
Dataframe 106 successfully save in database app-wsr in MongoDB. Time: --- 43.27333617210388 secnds ---
CPU times: user 24min 24s, sys: 14.5 s, total: 24min 38s
Wall time: 26min 17s


In [18]:
if USE_HEALTHCHECKS:
    step_2 = hc.done(hc_key)
    print(step_2)

Done ==> send to https://health.cashstory.com/ping/51c8f3e8-3d84-4888-878e-a53f0a152e1a, 2020-06-12
